# SetUp gRPC service with EC2

okay so the goal is to have an active gRPC service within an EC2 instance, and make service calls from a local client.


The main part to figure out is that since gRPC is serving via HTTP2, then it might be a bit different.
Also I need to practice the basics of working with AWS EC2 as well. so let's see...

okay the first step was to set up the simple grpc service and client that we need for testings.
I created the codes for that and  built it with docker and tested it.

in this effort one question that came up was that if I wanted to have an easy way to make this reproducable and easy to run like the way it is with docker, what would be the approach for the case of EC2? would it be possible or reasonable to install a docker inside the EC2
just to use for building/running an image? I guess not.

(BTW, [this](https://docs.aws.amazon.com/AmazonECS/latest/developerguide/docker-basics.html) shows how to install docker in an ec2)


well it apears to me that this work of service setup has two parts. one part in transfering and setting up the code in the EC2 and the other part is configuring the EC2 instance connection settings, etc. I'm going to first try to focus on the first part.

let's say I want to make an easy2run readme file here for all that is needed for ec2 to run. of course there are all those fancy k8s and pipelines to deploy the codes and images, but i want to stick to bare minimum. 

I guess it would start from git clone this repo.
oh wait assuming that git is available, which i guess is not, 
I guess maybe could start from yum to install git.

then python 3.7 is already installed. It should be good enough for this purpose, but what if I was really needing 3.8?

ok so I proceeded with the pip3 --user insttaltion of grpcio. but I get error

```
Command "python setup.py egg_info" failed with error code 1 in /tmp/pip-build-2mlrrjcs/grpcio/


 File "/usr/lib64/python3.7/subprocess.py", line 800, in __init__
        restore_signals, start_new_session)
      File "/usr/lib64/python3.7/subprocess.py", line 1551, in _execute_child
        raise child_exception_type(errno_num, err_msg, err_filename)
    FileNotFoundError: [Errno 2] No such file or directory: 'c++': 'c++'
```

maybe I should try to install python 3.8 here anyways?


okay so I install the python 3.8 from this 
https://techviewleo.com/how-to-install-python-on-amazon-linux/

then I run this :

```
python3.8 -m  pip install --user -r scripts/python_requirements.txt
```
or 

```
sudo python3.8 -m  pip install  -r scripts/python_requirements.txt
```

again I get the same error.


seems this issue is properly addressed here:

https://github.com/grpc/grpc/issues/24556

> I installed the Debian build-essentials meta-package as mentioned above. The packages that were missing for me were g++ and libstdc++


so how to install these?

okay looks like in order to install build-eesentials with yum  you need this 

https://serverfault.com/questions/204893/aws-ec2-and-build-essential/460295


but this was not enough to have the pip working.
let's try to instal g++ and libstdc++

https://www.fir3net.com/UNIX/Linux/how-do-i-install-g-via-yum.html




alright so the error in the installation of grpc in python was due to update g++ and libstdc++. nothing related to 3.8. so I can at least remove that part.


okay, starting with a fresh ec2, I run these :

```
sudo yum install gcc-c++
sudo yum install python3-devel
sudo pip3 install --upgrade setuptools

```

according to [this](https://github.com/grpc/grpc/issues/24556)

Re-visiting this issue after a few months...

> I installed the Debian build-essentials meta-package as mentioned above. The packages that were missing for me were g++ and libstdc++
After a couple of other updates (upgrading pip and setuptools) everything worked fine. Thanks for everyone's help ad input.


so looks like a rough equivalent of build-essential in yum is from [here](https://serverfault.com/questions/204893/aws-ec2-and-build-essential/460295)


```
yum groupinstall "Development Tools"
```

well still fails!
oaky so I next try to upgrade the pip itself

```
sudo pip3 install --upgrade pip
```

okay, now after that running the pip3 itself fails, since does not find the pip3. so for that I add it to the path.
export PATH="/usr/local/bin/pip3:$PATH"

and then this time it did actually install the "grpcio"



okay with all these I was able to run the server on the ec2

## Sending Regests to the grpc serverON EC2 from a local client

okay running the server on ec2, I wanted my client, i.e the test client on my local pc, to be able to send a request to it.

I know that the grpc uses the http2 protocol.

so it appears that it is realted to the security setting of ec2, and security groups.
So when defining the security groups, I was looking for an option that would seem specific for grpc, but I couldn't find it.
So in  the ec2 security settings I still tried adding http and https, but that does not work. So one option that was suggested [here](https://stackoverflow.com/questions/40027597/building-grpc-server-on-amazon-ec2) 
is to add a security group with "all trafic" option (of course just for a specific ip)
I tried that and it works. I was able to get my cool response!
"Greeter client received: Hello, Bob!"

but this does not seem to be the right way to do it, so at least now we get what is the issue.

# Setup the gRPC service with EC2 lunch type the Proper way!

okay so doing some search for this I found this article very useful:

https://dev.to/chaitan94/deploying-a-grpc-service-in-ecs-with-the-ec2-launch-type-2aa


(there were other stuff as well that kind of useful like [this](https://jitpaul.blog/2018/04/18/grpc-on-aws/) which is not addressing the part to actually send request to the server, but still is addressing the first part of this effort.)


the approach in this link, involves creating a container from the service, put it on ECR, create a task defenition (on ECS?)
and then  Setting up the target group and load balancer.
This tutorial though assumes that we already have a loadbalancer on ec2.
well which brings the question how to set up a load balancer?




## Useful links on gRPC service setup with AWS EC2

To My surprise there are not much of releavnt links that I could find on this topic on the internet. aprat from the one in above which is most relevant one, these are also related to this as well. I include all one more time for referece:

https://dev.to/chaitan94/deploying-a-grpc-service-in-ecs-with-the-ec2-launch-type-2aa

https://jitpaul.blog/2018/04/18/grpc-on-aws/

https://grpc.io/blog/grpc-load-balancing/

https://rokt.com/engineering_blog/learnings-grpc-aws/

https://kvwu.io/posts/set-up-grpc-with-aws/

https://aws.amazon.com/blogs/opensource/containerize-and-deploy-a-grpc-application-on-aws-fargate/

https://prefab-465416.hs-sites.com/blog/grpc-aws-some-gotchas

and last but not least 
https://stackoverflow.com/questions/50345084/deploy-grpc-supporting-application-on-aws-using-alb


## Set up and Load Balancer (and Target Groups)

okay so again going throught the tutorial, I faced the question "what is a load balancer and how to set it up?" 


so going through the aws tutorials did not give me anything but confusion
I found this tutorial to be helpfull for basic load balancer set up
https://www.youtube.com/watch?v=4gnMyhyVIg4

then with a bit of effort I could set up a clasic load balancer (the way described in the youtube video above)
which had an http listener.



But that was not exactly what specified in the tutorial. it wanted an LB with https listener.
So I tried to do that lb setup with https listener. I encounter the issue of certificates step in the set up.
There is the ACM option to create a certificate, but it was really confusing as to why is that even needed, and what is this all about?
plus going through the process of ACM certificate request, I relaized that I probably need to have a domain to be able to get that certificate. hmmm.
So from this point, I had to spend lots and lots of running around to know what is next step.

First I tried to see if I could setup the whole target group and assignment with the regular http LB, which was to no avail.
I tried with the whole target group defenition, and all the different things that should be set.
(One thing I realized in this was that the classic LB, does not include this target groups as there is in the new LBs for http and https. the one for https does not allow to include any target groups that has grpc protocol.)
so I guess no use to try with any LBs with http type listeners.

alright, going back to the https LB

## Set Up Load Balancer with HTTPS Listener

okay so it seemed that I needed to have a domain to make this https load balancer set up.
here I had two questions, how I can get a domain (for free or cheap) and second how does that domain even comes useful and relevant in the end-2-end process?

Again searching for these brings up a bunch of aws pages which was mostly confusing. I found this one helpfull for the second question at least:
https://www.youtube.com/watch?v=QA4X8ntcNBQ

going back to the first question ( a 20$ question for me :)

well looks like we need to have a domain to setup the alb with https listener.
the domain should allow for adding  certain CNAME record to the DNS configuration for your domain. 
so I needed some domain provider that is allowing that:
So far I found these three to be good options:

https://www.namecheap.com/ 

https://domains.google.com/

https://profreehost.com/


well I tried with the first one and proceeded with the process of ACM certificate issue. I did not go smoothly so I decided to go with the google one, just to realize that same issue will be faced.
Now I should cover a few points that was painfull to realize about the way to set up the certificate.
Here there

## Revise the greeter service and build service container

okay I set up the greeter servicer and greeter clients in seperate containers and ran a local test with a docker-compose 

## load the image to the ECR

first needed to set up the aws cli on my machine:
from here:
https://docs.aws.amazon.com/cli/latest/userguide/install-cliv2-mac.html


then onto setup the ECR account, etc:
from here:
https://blog.dbi-services.com/how-to-push-an-image-into-amazon-ecr-with-docker/


well did not work exactly as expetec:
need to set up an IAM account.
need to grant admin access.

usefull links:
https://docs.aws.amazon.com/AmazonECR/latest/userguide/getting-started-cli.html

https://docs.aws.amazon.com/AmazonECR/latest/userguide/getting-started-console.html

https://docs.aws.amazon.com/AmazonECR/latest/userguide/getting-started-cli.html

https://docs.aws.amazon.com/AmazonECR/latest/userguide/Registries.html#registry_auth

https://docs.aws.amazon.com/AmazonECR/latest/userguide/Registries.html#registry_auth

https://forums.aws.amazon.com/thread.jspa?threadID=310448


I was getting this error:

An error occurred (InvalidClientTokenId) when calling the GetCallerIdentity operation: The security token included in the request is invalid.

which was apparently because I had not setup the "aws cofigure" correctly.

we can set that up in the local machine aws (as well as in the web aws cli), but the local one is the one that I needed.
to properly do that we need to follow the instruction in the last link above.

going to this 
https://console.aws.amazon.com/iam/

and setup a permission policy and add the user to the policy and then generate access key ID and password.
then add those to the cli "Aws configure" prompts.

also noted that the instructions for tagging the docker container and pushing is actually for a private repository. for my case which was a public repo the correct commands was actually best found in the ecr repo auto generted codes:

```
aws ecr-public get-login-password --region us-east-1 | docker login --username AWS --password-stdin public.ecr.aws/p0c6a5h1

docker tag greeter_server:latest public.ecr.aws/p0c6a5h1/greeter_server:latest

docker push public.ecr.aws/p0c6a5h1/greeter_server:latest

```

okay now I have the server container in the ECR.